In [3]:
import dill
from cryptography.fernet import Fernet
import os
import time

def Singleton(cls):
    _instance = {}

    def _singleton(*args, **kwargs):
        if cls not in _instance:
            _instance[cls] = cls(*args, **kwargs)
        return _instance[cls]

    return _singleton

@Singleton
class A:
    a = 888

    def __init__(self, x=0):
        self.x = x
# 生成密钥并保存到文件
def generate_key():
    key = Fernet.generate_key()
    with open("secret.key", "wb") as key_file:
        key_file.write(key)

# 从文件加载密钥
def load_key():
    return open("secret.key", "rb").read()

# 加密数据
def encrypt(data, key):
    f = Fernet(key)
    return f.encrypt(data)

# 解密数据
def decrypt(data, key):
    f = Fernet(key)
    return f.decrypt(data)

# 序列化对象
def serialize_object(obj):
    return pickle.dumps(obj)

# 反序列化对象
def deserialize_object(data):
    return pickle.loads(data)

# 将对象保存到文件
def save_to_file(obj, filename, key):
    start_time = time.time()
    serialized_data = serialize_object(obj)
    serialization_time = time.time() - start_time

    start_time = time.time()
    encrypted_data = encrypt(serialized_data, key)
    encryption_time = time.time() - start_time

    start_time = time.time()
    with open(filename, 'wb') as f:
        f.write(encrypted_data)
    file_write_time = time.time() - start_time

    return serialization_time, encryption_time, file_write_time

# 从文件加载对象
def load_from_file(filename, key):
    start_time = time.time()
    with open(filename, 'rb') as f:
        encrypted_data = f.read()
    file_read_time = time.time() - start_time

    start_time = time.time()
    decrypted_data = decrypt(encrypted_data, key)
    decryption_time = time.time() - start_time

    start_time = time.time()
    obj = deserialize_object(decrypted_data)
    deserialization_time = time.time() - start_time

    return obj, file_read_time, decryption_time, deserialization_time

# 生成并加载密钥
if not os.path.exists("secret.key"):
    generate_key()
key = load_key()

# 场景模拟
def simulate():
    # 程序运行前
    a1 = A(888)
    print("Before shutdown:", a1)

    # 模拟程序关闭前保存状态
    serialization_time, encryption_time, file_write_time = save_to_file(a1, 'singleton.pkl', key)
    print(f"Serialization time: {serialization_time:.6f}s, Encryption time: {encryption_time:.6f}s, File write time: {file_write_time:.6f}s")

    # 清空实例以模拟程序关闭
    A._instance = {}

    # 程序启动时加载状态
    loaded_a1, file_read_time, decryption_time, deserialization_time = load_from_file('singleton.pkl', key)
    A._instance[A] = loaded_a1  # 恢复单例引用

    print("After startup:", loaded_a1)
    print(f"File read time: {file_read_time:.6f}s, Decryption time: {decryption_time:.6f}s, Deserialization time: {deserialization_time:.6f}s")



simulate()


Before shutdown: <__main__.A object at 0x7f65903c9ed0>
Serialization time: 0.000551s, Encryption time: 0.000311s, File write time: 0.000041s
After startup: <__main__.A object at 0x7f65903c9300>
File read time: 0.000011s, Decryption time: 0.000103s, Deserialization time: 0.000076s
